In [1]:
import torch
import pytubefix
import whisperx
import gc 
device = "cpu" 

batch_size = 16 # reduce if low on GPU mem
compute_type = "int8" # change to "int8" if low on GPU mem (may reduce accuracy)torch.serialization.default_load_weights_only = False


# 1. Transcribe with original whisper (batched)
model = whisperx.load_model("large-v2",device, compute_type=compute_type)

#model = whisperx.load_model("large-v2", "cpu")
print("Model loaded successfully")


INFO:speechbrain.utils.quirks:Applied quirks (see `speechbrain.utils.quirks`): [allow_tf32, disable_jit_profiling]
INFO:speechbrain.utils.quirks:Excluded quirks specified by the `SB_DISABLE_QUIRKS` environment (comma-separated list): []


No language specified, language will be first be detected for each audio file (increases inference time).


c:\Users\Katharina\AppData\Local\Programs\Python\Python39\lib\inspect.py:746: UserWarning: Module 'speechbrain.pretrained' was deprecated, redirecting to 'speechbrain.inference'. Please update your script. This is a change from SpeechBrain 1.0. See: https://github.com/speechbrain/speechbrain/releases/tag/v1.0.0
  if ismodule(module) and hasattr(module, '__file__'):
Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.5.0.post0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint c:\Users\Katharina\AppData\Local\Programs\Python\Python39\lib\site-packages\whisperx\assets\pytorch_model.bin`


Model was trained with pyannote.audio 0.0.1, yours is 3.3.2. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.6.0+cpu. Bad things might happen unless you revert torch to 1.x.
Model loaded successfully


In [2]:

link = "https://www.youtube.com/watch?v=3itfVgCN0sU"
yt = pytubefix.YouTube(link)

# Herunterladen des Audiostreams mit der höchsten verfügbaren Qualität
audio_stream = yt.streams.filter(only_audio=True).first()
mp3_file = "audio.mp3"

audio_stream.download(filename="temp_audio.m4a")

# Umwandlung von M4A zu MP3 (benötigt ffmpeg oder pydub)
from pydub import AudioSegment

audio = AudioSegment.from_file("temp_audio.m4a", format="m4a")
audio.export(mp3_file, format="mp3")

<_io.BufferedRandom name='audio.mp3'>

In [ ]:

audio = whisperx.load_audio(mp3_file)
result = model.transcribe(audio, batch_size=batch_size)
print(result["segments"]) # before alignment

# delete model if low on GPU resources
# import gc; gc.collect(); torch.cuda.empty_cache(); del model

# 2. Align whisper output
model_a, metadata = whisperx.load_align_model(language_code=result["language"], device=device)
result = whisperx.align(result["segments"], model_a, metadata, audio, device, return_char_alignments=False)

print(result["segments"]) # after alignment

# delete model if low on GPU resources
# import gc; gc.collect(); torch.cuda.empty_cache(); del model_a

# 3. Assign speaker labels
diarize_model = whisperx.DiarizationPipeline(use_auth_token=hf_access_token, device=device)

# add min/max number of speakers if known
diarize_segments = diarize_model(audio)
# diarize_model(audio, min_speakers=min_speakers, max_speakers=max_speakers)

result = whisperx.assign_word_speakers(diarize_segments, result)
print(diarize_segments)
print(result["segments"]) # segments are now assigned speaker IDs



Detected language: de (1.00) in first 30s of audio...


In [18]:
for segment in result["segments"]:
    for word in segment["words"]:
        print(f"{word['text']} - Start: {word['start']}, Ende: {word['end']}")

NameError: name 'result' is not defined

In [ ]:
result["text"]

In [ ]:
# Extract the transcribed text
transcribed_text = result["text"]

with open("transcription_" + name + ".txt", "w", encoding="utf-8") as file:
    file.write(transcribed_text)

In [ ]:
# Merge speaker diarization and transcription
diarization_segments = []
min_duration = 1.0  #Minimum duration for a speach in seconds
tolerance = 0.2  # Allows small deviations for the first words

for segment, _, speaker in diarization.itertracks(yield_label=True):
    segment_duration = segment.end - segment.start
    if segment_duration >= min_duration:
        diarization_segments.append((segment.start, segment.end, speaker))

active_speaker = None
combined_output = []
merged_start = None
merged_end = None
merged_speaker = None
merged_text = []

# Check which speaker is assigned to each word in the transcription
for segment in result["segments"]:
    for word in segment["words"]:
        current_time = word["start"]

        # Find the matching speaker segment based on the timestamps
        closest_speaker = None
        for start, end, speaker in diarization_segments:
            # Check whether the word is within a segment or just before it (tolerance)
            if start - tolerance <= current_time <= end:
                closest_speaker = speaker
                break 

        # When a speaker has been found
        if closest_speaker is not None:
            # If the speaker changes or no speaker is set yet
            if active_speaker != closest_speaker:
                # If there has been a change of speaker, write the previous text
                if merged_speaker:
                    speaker_name = speaker_names.get(merged_speaker, "Unknown Speaker")
                    combined_output.append(f"{speaker_name}: {' '.join(merged_text)}")
                
                # Set variables for the new speaker
                merged_speaker = closest_speaker
                merged_start = word["start"]
                merged_end = word["end"]
                merged_text = [word["text"]]
                active_speaker = closest_speaker  # Update active speaker
            else:
                # If the speaker remains the same, add the word
                merged_end = word["end"]
                merged_text.append(word["text"])
        else:
            # If no speaker can be assigned, append the word to the last speaker
            if merged_speaker:
                merged_text.append(word["text"])

# Add last merged speaker, if present
if merged_speaker:
    speaker_name = speaker_names.get(merged_speaker, "Unknown Speaker")
    combined_output.append(f"{speaker_name}: {' '.join(merged_text)}")

# Save output
output_file = "combined_output" + name + ".txt"
with open(output_file, "w", encoding="utf-8") as f:
    for line in combined_output:
        f.write(line + "\n")